In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import configparser
import os.path
from os import path
from importlib import reload

creds_path_ar = ["../../credentials.ini","credentials.colab.ini"]
PATH_ROOT = ""
PATH_DATA = ""

for creds_path in creds_path_ar:
    if path.exists(creds_path):
        config_parser = configparser.ConfigParser()
        config_parser.read(creds_path)
        PATH_ROOT = config_parser['MAIN']["PATH_ROOT"]
        PATH_DATA = config_parser['MAIN']["PATH_DATA"]
        WANDB_enable = config_parser['MAIN']["WANDB_ENABLE"] == 'TRUE'
        ENV = config_parser['MAIN']["ENV"]
        break

if ENV=="COLAB":
  from google.colab import drive
  mount_path = '/content/gdrive/'
  drive.mount(mount_path)

Drive already mounted at /content/gdrive/; to attempt to forcibly remount, call drive.mount("/content/gdrive/", force_remount=True).


In [3]:
if WANDB_enable == True:
    if ENV=="COLAB":
      !pip install --upgrade wandb
    import wandb
    !wandb login {config_parser['MAIN']["WANDB_LOGIN"]}
    wandb.init(project="sota-mafat-base")
    os.environ['WANDB_NOTEBOOK_NAME'] = '[SS]Alexnet_pytorch'

In [4]:
cd {PATH_ROOT}

/content/gdrive/My Drive/WORK/ML/develop/MAFAT/sota-mafat-radar


In [5]:
import logging

log_filename = "alexnet_pytorch.log"
if os.path.exists(log_filename):
    os.remove(log_filename)

logging.basicConfig(level=logging.INFO,
                    filename='alexnet_pytorch.log',
                    format="%(asctime)s [%(levelname)s]|%(module)s:%(message)s",)

logging.info("start")
logger = logging.getLogger()


In [6]:
import os
import random
import pickle
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
 
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim
 
from sklearn.metrics import roc_auc_score, roc_curve, auc, accuracy_score
from matplotlib.colors import LinearSegmentedColormap
from termcolor import colored
 
from src.data import feat_data, get_data, get_data_pipeline
from src.models import arch_setup, alex_model, dataset_ram_reduced
from src.features import specto_feat,add_data
 
# Set seed for reproducibility of results
seed_value = 0
os.environ['PYTHONHASHSEED']=str(seed_value)
 
 
random.seed(seed_value)
np.random.seed(seed_value)
torch.manual_seed(0)
 
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
 
if torch.cuda.is_available():
    device = torch.device('cuda:0')
else:
    device = torch.device('cpu:0')

In [7]:
config = dict()
config['num_tracks'] = 0
config['val_ratio'] = 3
config['shift_segment'] = list(np.arange(1,31))
config['get_shifts'] = False
config['get_horizontal_flip'] = False
config['get_vertical_flip'] = False

batch_size = 32
lr = 1e-4

In [14]:
full_data_picklefile = PATH_DATA+'/full_data.pickle'

if path.exists(full_data_picklefile):
  print('getting full_data from pickle')
  with open(full_data_picklefile, 'rb') as handle:
      full_data = pickle.load(handle)

else:
  full_data = get_data_pipeline.pipeline_trainval_ram_reduced(PATH_DATA, config)
  ## SAVE TO PICKLE
  with open(full_data_picklefile, 'wb') as handle:
      pickle.dump(full_data, handle,protocol=pickle.HIGHEST_PROTOCOL)

In [15]:
print(len(full_data[full_data.is_validation==False]))
print(len(full_data[full_data.is_validation==True]))


6040
616


In [ ]:
train_set = dataset_ram_reduced.DS(full_data[full_data.is_validation==False])
val_set= dataset_ram_reduced.DS(full_data[full_data.is_validation==True])

train_y = np.array(full_data[full_data.is_validation==False]['target_type']=='human').astype(int)
val_y = np.array(full_data[full_data.is_validation==True]['target_type']=='human').astype(int)

train_loader=DataLoader(dataset= train_set, batch_size = batch_size, shuffle = True, num_workers = 2)
val_loader=DataLoader(dataset= val_set, batch_size = batch_size, shuffle = True, num_workers = 2)

In [ ]:
model= alex_model.alex_mdf_model()
# model.apply(init_weights)
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)

model.to(device)

if WANDB_enable == False:
  wandb = None
else:
    runname = input("Enter WANDB runname(ENTER to skip wandb) :")
    notes = input("Enter run notes :")

    wandb.init(project="sota-mafat-base",name=runname, notes=notes)
    os.environ['WANDB_NOTEBOOK_NAME'] = '[SS]Alexnet_pytorch'
    
    wandb.watch(model)
    wandb.config['data_config'] = config
    wandb.config['train_size'] = len(full_data[full_data.is_validation==False])
    wandb.config['val_size'] = len(full_data[full_data.is_validation==True])
    wandb.config['batch_size'] = batch_size
    wandb.config['learning rate'] = lr
    wandb.log(config)


Enter WANDB runname(ENTER to skip wandb) :ram reduced batch32
Enter run notes :


wandb: Waiting for W&B process to finish, PID 643
wandb: Program ended successfully.


wandb:                                                                                
wandb: Find user logs for this run at: wandb/run-20200922_205603-1g3glb9f/logs/debug.log
wandb: Find internal logs for this run at: wandb/run-20200922_205603-1g3glb9f/logs/debug-internal.log
wandb: Synced 5 W&B file(s), 0 media file(s), 0 artifact file(s) and 0 other file(s)
wandb: 
wandb: Synced light-dragon-48: https://wandb.ai/sota-mafat/sota-mafat-base/runs/1g3glb9f
wandb: Tracking run with wandb version 0.10.2
wandb: Run data is saved locally in wandb/run-20200922_205702-10nvxn6o
wandb: Syncing run ram reduced batch32


In [ ]:
log = arch_setup.train_epochs(train_loader,val_loader,model,criterion,optimizer,num_epochs= 10,device=device,train_y=train_y,val_y=val_y, WANDB_enable = WANDB_enable, wandb= wandb)

started training epoch no. 1



epoch : 1.0000, loss : 932.3602, auc : 0.9903, acc : 0.9751, val_loss : 7.6302, val_auc : 0.9510, val_acc : 0.8766, 
---------------------------

started training epoch no. 2



epoch : 2.0000, loss : 339.5792, auc : 0.9986, acc : 0.9920, val_loss : 11.4702, val_auc : 0.9381, val_acc : 0.8588, 
---------------------------

started training epoch no. 3



epoch : 3.0000, loss : 204.7947, auc : 0.9993, acc : 0.9958, val_loss : 14.0210, val_auc : 0.9539, val_acc : 0.8896, 
---------------------------

started training epoch no. 4



epoch : 4.0000, loss : 152.7131, auc : 0.9996, acc : 0.9969, val_loss : 19.4269, val_auc : 0.9567, val_acc : 0.8945, 
---------------------------

started training epoch no. 5


...


epoch : 6.0000, loss : 144.3722, auc : 0.9996, acc : 0.9977, val_loss : 18.0621, val_auc : 0.9470, val_acc : 0.8815, 
---------------------------

started training epoch no. 7



epoch : 7.0000, loss : 118.5509, auc : 0.9998, acc : 0.9979, val_loss : 9.9136, val_auc : 0.9520, val_acc : 0.8896, 
---------------------------

started training epoch no. 8



epoch : 8.0000, loss : 139.1546, auc : 0.9998, acc : 0.9980, val_loss : 38.4082, val_auc : 0.9497, val_acc : 0.9010, 
---------------------------

started training epoch no. 9



epoch : 9.0000, loss : 174.5771, auc : 0.9996, acc : 0.9978, val_loss : 32.6661, val_auc : 0.9505, val_acc : 0.9026, 
---------------------------

started training epoch no. 10



epoch : 10.0000, loss : 131.3528, auc : 0.9997, acc : 0.9980, val_loss : 30.1615, val_auc : 0.9543, val_acc : 0.9026, 
---------------------------



In [ ]:
pred = [model(torch.from_numpy(train_x).to(device, dtype=torch.float)).detach().cpu().numpy(),
        model(torch.from_numpy(val_x).to(device, dtype=torch.float)).detach().cpu().numpy()]
actual = [train_y, val_y]
metrics.stats(pred, actual)

## SUBMIT

In [ ]:
test_path = 'MAFAT RADAR Challenge - Public Test Set V1'
test_df = get_data.load_data(test_path, PATH_DATA)
test_df = specto_feat.data_preprocess(test_df.copy())
test_x = test_df['iq_sweep_burst']
test_x = test_x.reshape(list(test_x.shape)+[1])

FileNotFoundError: ignored

In [ ]:
# Creating DataFrame with the probability prediction for each segment
submission =  pd.DataFrame()
submission['segment_id'] = test_df['segment_id']
submission['prediction'] = model(torch.from_numpy(test_x).to(device).type(torch.float32)).detach().cpu().numpy()
submission['prediction'] = submission['prediction'].astype('float')

# Save submission
submission.to_csv('submission.csv', index=False)